    classifiers used

        regression
            linear regression >> with polyfeatures
            logistic regression
            lasso regression
            ridge regression

        knn
        desicion tree
        svm
        naivebias

        ensemble methods:
            bagging
                random forest 

            boosting
                gradient boost  
                ada boost
                xg boost
                cat boost


        hyperparameter tuning(depending on model):
            grid search


        evaluation and validation :
            cross validation
            ROC-AUC curve
            r2 score
            confuction matrix

## imports

In [9]:
import sklearn as sk
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
from IPython.display import display


In [10]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import numpy as np
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.preprocessing import normalize
from sklearn.svm import SVC


## helper functions for plotting and visualization


In [52]:
# helper functions for plotting and visualization

#  for plotting confusion matrix
def plot_confusion_matrix(model, x_test, y_test):
    cm = confusion_matrix(y_test, model.predict(x_test))
    # cm_norm = cm / cm.sum(axis=1).reshape(-1,1)
    score = model.score(x_test, y_test)
    plt.figure(figsize=(9, 9))
    sns.heatmap(cm, annot=True, fmt=".1f", linewidths=.5,
                square=True, cmap='Blues_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    all_sample_title = 'Accuracy Score: {0}'.format(score)
    plt.title(all_sample_title, size=15)


# for printing the precision , recall , f1-score , ROC(AOC) , classification report
def print_metrics(model, x_train, x_test, y_train, y_test):
    print("The Training Accuracy is: ", model.score(x_train, y_train))
    print("The Testing Accuracy is: ", model.score(x_test, y_test))
    y_pred = model.predict(x_test)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    score = f1_score(y_test, y_pred, average='binary')
    ROC_AUC = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
    print('Precision: %.3f' % precision)
    print('Recall: %.3f' % recall)
    print('F1-Measure: %.3f' % score)
    print('ROC_AUC: %.3f' % ROC_AUC)
    print(
        f'the classification report is \n {classification_report(y_test, y_pred)}')


# for plotting the ROC curve (takes the model and test data as inputs)
def plot_roc(model, name,  x_test, y_test):
    pred_prob = model.predict_proba(x_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred_prob[:, 1])

    # roc curve for tpr = fpr
    random_probs = [0 for i in range(len(y_test))]
    p_fpr, p_tpr, _ = roc_curve(y_test, random_probs)

    auc_score = auc(fpr, tpr)

    plt.figure(figsize=(10, 10))
    plt.style.use('seaborn')

    plt.plot(fpr, tpr, linestyle='--', color='green',
             label=f'{name} ROC (AUC = %0.4f)' % auc_score)
    plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
    plt.legend(loc='lower right')
    plt.title("ROC Curve")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive rate')
    plt.show()


## data preprocessing

In [12]:
import glob
path = glob.glob(r'./IBM/*.csv')
path

['./IBM\\WA_Fn-UseC_-HR-Employee-Attrition.csv']

In [13]:
df = pd.read_csv(path[0])

In [58]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [54]:
# print(df.info())
# display(df.describe())

In [63]:
print('size of the original dataframe is: ', df.shape)

size of the original dataframe is:  (1470, 35)


In [20]:
print('max perf' , df['PerformanceRating'].max())
print('min perf',df['PerformanceRating'].min())
# print(df['PerformanceRating'].head())
print('descrete values of perf' , set(df['PerformanceRating'].to_numpy()))


max perf 4
min perf 3
descrete values of perf {3, 4}


In [55]:
print(df['Attrition'].head())


0    1
1    0
2    1
3    0
4    0
Name: Attrition, dtype: int64


In [60]:
df['Attrition'].replace(['Yes', 'No'], [1, 0], inplace=True)
print(df['Attrition'].head().to_markdown())


|    |   Attrition |
|---:|------------:|
|  0 |           1 |
|  1 |           0 |
|  2 |           1 |
|  3 |           0 |
|  4 |           0 |


In [61]:
x = np.array(df.drop(['Attrition', 'PerformanceRating'], axis=1))
y = np.array(df[['Attrition', 'PerformanceRating']])
print('columns of X , Y ',x.shape[1], y.shape[1])
print('rows of X , Y ',x.shape[0] , y.shape[0])


columns of X , Y  33 2
rows of X , Y  1470 1470


In [49]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)
length = len(x_train)


array([[41, 'Travel_Rarely', 1102, ..., 4, 0, 5],
       [49, 'Travel_Frequently', 279, ..., 7, 1, 7],
       [37, 'Travel_Rarely', 1373, ..., 0, 0, 0],
       ...,
       [27, 'Travel_Rarely', 155, ..., 2, 0, 3],
       [49, 'Travel_Frequently', 1023, ..., 6, 0, 8],
       [34, 'Travel_Rarely', 628, ..., 3, 1, 2]], dtype=object)

## various models training , tuning and Evaluation

In [50]:
k_values = [i for i in range(1, 10)]
k_acc_scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    cv_scores = cross_val_score(knn, x, y, cv=10, scoring="recall")
    k_acc_scores.append(cv_scores.mean())


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 400, in _fit
    X, y = self._validate_data(X, y, accept_sparse="csr", multi_output=True)
  